In [1]:
import sys
import os

# Add the project root to sys.path
sys.path.append(os.path.abspath(os.path.join(os.path.abspath(''), "..")))

In [2]:
from src.dataset import EpicKitchens100Dataset
from src.config import DevConfig

In [3]:
dataset = EpicKitchens100Dataset(DevConfig.ROOT_DIR, DevConfig.ANNOTATIONS_DIR_RELATIVE)
dataset[0]["frames"]

tensor([[[[ 3,  3,  3,  ...,  4,  4,  4],
          [ 3,  3,  3,  ...,  4,  4,  4],
          [ 3,  3,  3,  ...,  4,  4,  4],
          ...,
          [ 3,  3,  3,  ...,  1, 12, 21],
          [ 3,  3,  3,  ..., 12, 30, 45],
          [ 3,  3,  3,  ..., 21, 44, 62]],

         [[ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          ...,
          [ 0,  0,  0,  ...,  0, 10, 19],
          [ 0,  0,  0,  ..., 10, 28, 43],
          [ 0,  0,  0,  ..., 19, 42, 60]],

         [[ 4,  4,  4,  ...,  3,  3,  3],
          [ 4,  4,  4,  ...,  3,  3,  3],
          [ 4,  4,  4,  ...,  3,  3,  3],
          ...,
          [ 2,  2,  2,  ...,  2, 13, 22],
          [ 2,  2,  2,  ..., 13, 31, 46],
          [ 2,  2,  2,  ..., 22, 45, 63]]],


        [[[ 3,  3,  3,  ...,  5,  5,  5],
          [ 3,  3,  3,  ...,  5,  5,  5],
          [ 3,  3,  3,  ...,  5,  5,  5],
          ...,
          [ 3,  3,  3,  ..., 65, 73, 60],
        

In [4]:
dataset[0]['verb_class']

np.int64(3)

In [5]:
dataset[35]["frames"].shape

torch.Size([112, 3, 256, 456])

#### Check transforms

In [8]:
from torchvision.transforms import Compose, Lambda
from torchvision.transforms import Normalize, CenterCrop
from src.transforms import FixedSizeClipSampler, TransformKey, PackPathway

mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
train_transform = Compose(
    [
        FixedSizeClipSampler(num_frames=32),
        Lambda(lambda x: x / 255.0),
        Normalize(mean, std),
        CenterCrop(crop_size),
        Lambda(lambda x: x.permute(1, 0, 2, 3)),
        PackPathway()
    ]
)
train_transform = TransformKey("frames", train_transform)

In [9]:
dataset_transformed = EpicKitchens100Dataset(r"C:\Users\Jani\EPIC-KITCHENS", r"annotations\train_till_P105.csv", transform=train_transform)

In [14]:
dataset_transformed[0]["frames"][0].shape, dataset_transformed[0]["frames"][1].shape

(torch.Size([3, 8, 256, 256]), torch.Size([3, 32, 256, 256]))

In [15]:
import torch

model = torch.hub.load("facebookresearch/pytorchvideo", "slowfast_r50", pretrained=True)

Using cache found in C:\Users\Jani/.cache\torch\hub\facebookresearch_pytorchvideo_main


In [17]:
model.blocks

ModuleList(
  (0): MultiPathWayWithFuse(
    (multipathway_blocks): ModuleList(
      (0): ResNetBasicStem(
        (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
      (1): ResNetBasicStem(
        (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
        (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
        (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
      )
    )
    (multipathway_fusion): FuseFastToSlow(
      (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), stride=(4, 1, 1), padding=(3, 0, 0), bias=False)
    